# Import

In [6]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from main import backgammon

# Import datasets

In [ ]:
def load_dataset(file_path):
    # Load CSV using pandas
    df = pd.read_csv(file_path)
    
    # Split into features and labels
    data = df.values
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure shape (N, 1)
    
    return TensorDataset(X, y)

train_file = os.path.join("..","Data","Deep","RSP",'train.txt')
# validation_file = os.path.join("..","Data","Deep","RSP",'validation.txt')
test_file = os.path.join("..","Data","Deep","RSP",'test.txt')

train_dataset = load_dataset(train_file)
# validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Example: iterate over a batch from the train set
for X_batch, y_batch in train_loader:
    print("Input shape:", X_batch.shape)
    print("Labels shape:", y_batch.shape)
    break

Input shape: torch.Size([64, 578])
Labels shape: torch.Size([64, 1])


In [8]:
## BGNET HERE ##
class BGNet(nn.Module):
    def __init__(self):
            super(BGNet, self).__init__()
            self.fc1 = nn.Linear(289, 12)  # Fully connected input → hidden
            self.fc2 = nn.Linear(12, 1)    # Fully connected hidden → output

    def forward(self, x):
        x = torch.tanh(self.fc1(x))        # Apply tanh to hidden layer
        x = self.fc2(x) 
        return x

## Training loop

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assume: model is your instance of BGNet
model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training settings
num_epochs = 100  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch[:, :289])
            loss = criterion(outputs.squeeze(), y_batch.squeeze())
            total_loss += loss.item() * y_batch.size(0)

            # preds = outputs.float()
            # correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    
    avg_loss = total_loss / total
    # accuracy = correct / total
    return avg_loss#, accuracy

# === Training Loop ===
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        board = X_batch[:, :289]

        outputs = model(board)
        loss = criterion(outputs.squeeze(), y_batch.squeeze())

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * y_batch.size(0)

        # For training accuracy
        # preds = (outputs > 0.5).float()
        # correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_train_loss = running_loss / total
    # train_acc = correct / total

    if epoch % 10 == 0:
        test_loss = evaluate(model, test_loader)
        
        print(f"Epoch {epoch:03d}:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Test  Loss: {test_loss:.4f}\n")

    torch.save(model.state_dict(), "RSPbackgammon_model.pth")


Epoch 010:
  Train Loss: nan
  Test  Loss: nan

Epoch 020:
  Train Loss: nan
  Test  Loss: nan

Epoch 030:
  Train Loss: nan
  Test  Loss: nan

Epoch 040:
  Train Loss: nan
  Test  Loss: nan

Epoch 050:
  Train Loss: nan
  Test  Loss: nan

Epoch 060:
  Train Loss: nan
  Test  Loss: nan

Epoch 070:
  Train Loss: nan
  Test  Loss: nan

Epoch 080:
  Train Loss: nan
  Test  Loss: nan

Epoch 090:
  Train Loss: nan
  Test  Loss: nan

Epoch 100:
  Train Loss: nan
  Test  Loss: nan

